<a href="https://colab.research.google.com/github/bhushanyadav07/Build-A-Custom-Sentiment-Analyzer/blob/master/Sentiment_Analyzer_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import nltk
nltk.download('popular')

[nltk_data]    | Downloading package stopwords to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/stopwords.zip.
[nltk_data]    | Downloading package treebank to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/treebank.zip.
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/twitter_samples.zip.
[nltk_data]    | Downloading package omw to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/omw.zip.
[nltk_data]    | Downloading package wordnet to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/wordnet.zip.
[nltk_data]    | Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/wordnet_ic.zip.
[nltk_data]    | Downloading package words to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/words.zip.
[nltk_data]    | Downloading package maxent_ne_chunker to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping chunkers/maxent_n

True

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [16]:
import pandas as pd
df = pd.read_csv('/content/movie_review_data_1000.csv')
df.head()

,review,sentiment
0,JESSICA: A GHOST STORY is as the name implies ...,0
1,After seeing Arthur on TV numerous times I lau...,1
2,"Set in Venice mainly on the Lido, Visconti's ""...",1
3,Morris and Reva Applebaum had been the toast o...,1
4,"Based on fact, this is the story of a teenager...",1


Let's split the dataset into training dataset and test dataset

In [0]:
X_train = list(df.loc[:499, 'review'].values)
X_test = list(df.loc[500:, 'review'].values)


Gensim library has Doc2Vec model for document embedding.

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(X_train)]


In [19]:
max_epochs = 50
vec_size = 100
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
        alpha=alpha, 
        min_alpha=0.00025,
        min_count=1,
        dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
  model.train(tagged_data,
        total_examples=model.corpus_count,
        epochs=model.epochs)
  model.alpha -= 0.0002
  model.min_alpha = model.alpha

model.save("doc.vec")
print("Model Saved")

Model Saved


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [20]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("doc.vec")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import numpy as np

narr = np.append([], [])
for text in X_train:
  txt = word_tokenize(text.lower())
  vect = model.infer_vector(txt)
  narr = np.append(narr, vect)

In [22]:
for text in X_test:
  txt = word_tokenize(text.lower())
  vect = model.infer_vector(txt)
  narr = np.append(narr, vect)
narr = np.resize(narr, (1000, vec_size))
narr.shape

(1000, 100)

In [23]:
df_trans = pd.DataFrame(narr, columns=['col_'+ str(i+1) for i in range(narr.shape[1])])
df_trans['sentiment'] = df['sentiment']
df_trans.to_csv('review_1000_doc2_vectors.csv', index=False)
df_trans.tail()

,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,...,col_62,col_63,col_64,col_65,col_66,col_67,col_68,col_69,col_70,col_71,col_72,col_73,col_74,col_75,col_76,col_77,col_78,col_79,col_80,col_81,col_82,col_83,col_84,col_85,col_86,col_87,col_88,col_89,col_90,col_91,col_92,col_93,col_94,col_95,col_96,col_97,col_98,col_99,col_100,sentiment
995,0.008487,0.079526,0.527572,0.172163,0.640410,-0.775763,-0.048436,0.258563,-0.039940,0.200585,-0.127049,-0.066882,0.705558,-0.013112,-0.748331,-0.484203,-0.870236,-0.719877,0.686585,-0.073148,0.397806,-0.693636,-0.207654,0.019975,-0.043854,0.737216,-0.273398,0.311398,0.344621,0.387629,-0.273294,-0.010320,-0.145136,0.171045,-0.397636,-0.278942,0.281367,-0.395643,0.188270,0.053546,...,-0.169895,0.149734,-0.355726,-0.414885,0.754072,-0.567409,-0.480481,0.034636,0.218084,0.411720,-0.256951,0.700536,0.249572,-0.287817,0.114178,-0.103386,0.554211,-0.382643,0.504525,0.201828,-0.135540,0.118841,-0.017539,-0.280543,0.350820,-0.162952,0.704203,-0.247442,0.844109,0.037481,0.607443,-0.108885,0.627978,-0.447699,-0.490412,0.119808,0.650508,-0.492264,0.254809,1
996,-0.002679,-0.337997,1.225427,0.768010,0.602961,-0.208625,0.033261,-0.672800,-0.780016,-0.186654,-1.127952,0.621866,-0.390013,-1.021757,-0.777816,-0.474770,-0.906056,-0.730499,1.032141,-0.395588,2.031873,0.223388,0.114764,0.524933,-1.054202,0.644982,1.354288,-0.906464,1.292914,0.262477,-0.253718,0.113492,-0.328531,0.029236,0.407831,-0.041241,2.268480,0.439449,-0.566568,0.169786,...,0.124391,-0.624758,0.289427,0.721704,0.201664,0.193589,0.839057,-0.463120,0.008472,-0.162209,-2.038005,1.631191,-0.042666,-0.095576,-0.203044,-0.201293,2.003690,0.537425,-1.024035,-0.153579,-1.043413,-0.468102,-1.593666,-0.450425,0.707788,-0.453558,0.584284,0.874617,0.321218,0.088861,0.561305,0.475657,0.346379,-1.126152,-0.807383,-0.529319,-0.593381,0.017134,0.606417,1
997,-0.475991,0.220041,0.471021,-0.309271,0.478046,-0.041502,0.520249,-0.334986,0.255322,-0.535653,-0.472607,0.016013,0.106733,0.034810,0.805805,0.354120,0.199057,-0.912593,0.077709,-0.392638,0.351412,0.022783,-0.042694,0.053353,-0.441590,-0.388711,0.009425,-0.231858,0.096757,-0.029168,-0.193022,-0.479313,0.657090,-0.355939,1.112838,-1.591200,0.888830,0.708414,0.461422,1.020317,...,-0.193264,0.770189,0.558394,-0.355507,-0.807153,-1.740780,0.699648,1.043610,2.194575,0.222528,-0.020964,-0.564139,-0.068622,-0.692367,-0.637701,-0.086968,0.731171,1.161714,-0.410878,-0.201156,0.657999,1.457757,0.070641,-0.042812,-0.501968,-0.849059,1.016083,-0.734586,0.869125,-0.193323,1.679061,0.922826,-0.034835,-0.702982,-0.298118,1.508353,-0.035963,-0.415822,-0.234750,0
998,-0.571714,0.241518,0.681031,0.141302,-0.226776,-0.153680,-0.154102,-0.344304,-0.452006,-0.961428,-0.328265,0.267817,0.827572,-0.156678,-0.030250,-0.125698,0.351487,-0.854505,1.190361,-0.057365,0.626629,0.150477,-0.538803,0.710019,0.252820,0.006028,0.053655,-0.666364,-0.082249,-0.295824,0.608302,-0.452764,-0.041747,0.600114,-0.292960,0.027518,0.187527,0.167443,0.440254,0.511428,...,0.273914,0.027703,0.430310,0.439048,-0.318901,-0.518013,0.471331,0.974279,0.838721,-0.097484,0.255851,0.252106,0.158562,-0.392854,-0.492787,0.181744,0.706151,0.110601,0.804278,0.151898,0.260092,-0.425573,-0.500423,0.019612,0.425121,-0.236941,-0.303424,-0.615122,0.901163,-0.218255,-0.305712,-0.139697,0.304115,-0.238146,0.105093,0.381665,0.709164,-0.399191,0.363554,1
999,-0.772244,-0.472136,0.899859,0.066992,0.452649,0.507661,-0.377624,-0.637012,1.292334,0.240784,-0.334342,-0.600861,0.008263,0.209382,-1.308038,-0.485698,0.247770,0.319897,0.439984,-0.520316,0.312119,0.078693,-1.188705,-0.686394,-1.392790,-0.354136,0.018104,-0.508660,-0.906005,-0.234550,-0.130826,-1.354116,0.237870,-0.551710,-0.464636,0.187709,0.856280,-0.840335,-0.601130,-1.295240,...,0.285546,0.2